In [3]:
import pandas as pd

In [73]:
df = pd.read_csv('e:/data/OAPMLData/CAIC_Forecasts.csv', 
                 parse_dates=['date', 'date_modified', 'date_issued'],
                 dtype = {'rating': 'object', 
                          'aspect_elev_0': 'object', 'aspect_elev_1': 'object', 'aspect_elev_2': 'object', 
                          'size_0': 'object', 'size_1': 'object', 'size_2': 'object',
                          'problem_0': 'object', 'problem_1': 'object', 'problem_2': 'object'})

In [74]:
#1. filter out draft
df = df[df['status']=='published']


In [75]:
df.head()

,bc_avo_fx_id,status,zone_id,user_id,date,rating,summary,tweet,problems,problem_0,...,zone_id.1,lat,lon,utm_zone,utm_e,utm_n,name,title,anchor,zone_order
0,2,published,6,17,2013-12-23 16:14:41,021021,<p>The Grand Mesa picked has picked up NUMBERS...,NaN,2.0,3,...,6.0,39.07342,-107.97518,13.0,242616.0,4329139.0,Grand Mesa,Grand Mesa,grandmesa,6.0
1,3,published,9,17,2013-12-23 16:18:25,000000,<p>Remote weather stations are showing a coupl...,NaN,1.0,4,...,9.0,37.73237,-105.33148,13.0,470792.0,4176173.0,Sangre de Cristo,Sangre de Cristo Mountains,sangre,9.0
2,4,published,0,17,2013-12-23 16:20:59,332332,<p>Snowfall continues in the Steamboat and Fla...,NaN,2.0,3,...,0.0,40.45217,-106.66901,13.0,358475.0,4479283.0,Steamboat & Flat Tops,Steamboat Area & Flat Tops,steamboat,0.0
3,5,published,1,17,2013-12-23 16:24:09,331331,<p>Overnight 1 to 3 inches of new snow fell in...,NaN,2.0,3,...,1.0,40.14328,-105.84075,13.0,428384.0,4443999.0,Front Range,Front Range,frontrange,1.0
4,6,published,2,17,2013-12-23 16:27:19,331331,<p>Dangerous avalanche conditions exist. Avala...,NaN,2.0,3,...,2.0,39.59163,-106.07693,13.0,407527.0,4382987.0,Vail & Summit County,Vail & Summit County,vailsummit,2.0


In [76]:
def number_to_danger(num):
    if num == '0':
        return 'no-data'
    elif num == '1':
        return 'Low'
    elif num == '2':
        return 'Moderate'
    elif num == '3':
        return 'Considerable'
    elif num == '4':
        return 'High'
    elif num == '5':
        return 'Extreme'
    else:
        return 'Unknown-Danger'
    
    

In [77]:
df['Day1DangerAboveTreeline'] = df['rating'].str[0].apply(number_to_danger)
df['Day1DangerNearTreeline'] = df['rating'].str[1].apply(number_to_danger)
df['Day1DangerBelowTreeline'] = df['rating'].str[2].apply(number_to_danger)
df['Day2DangerAboveTreeline'] = df['rating'].str[3].apply(number_to_danger)
df['Day2DangerNearTreeline'] = df['rating'].str[4].apply(number_to_danger)
df['Day2DangerBelowTreeline'] = df['rating'].str[5].apply(number_to_danger)

In [78]:
import numpy as np
def num_to_problem(num):
    if num == '0':
        return 'LooseDry'
    elif num == '1':
        return 'LooseWet'
    elif num == '2':
        return 'StormSlabs'
    elif num == '3':
        return 'WindSlab'
    elif num == '4':
        return 'PersistentSlab'
    elif num == '5':
        return 'DeepPersistentSlab'
    elif num == '6':
        return 'WetSlabs'
    elif num == '7':
        return 'Cornices'
    elif num == '8':
        return 'Glide'
    else:
        raise Exception('Unknown Problem Exception with num: ' + str(num))

def num_to_likelihood(num):
    if num == 0:
        return '0-unlikely'
    elif num == 1:
        return '1-possible'
    elif num == 2:
        return '2-likely'
    elif num == 3:
        return '3-very likely'
    elif num == 4:
        return '4-certain'
    else:
        raise Exception('Unknown Likelihood Exception with num: ' + str(num))

def str_to_maximum_size(s):
    if(pd.isna(s)):
        return 'no-data'
    
    if s[0] == '1':
        return '3-historic'
    elif s[1] == '1':
        return '2-very large'
    elif s[2] == '1':
        return '1-large'
    elif s[3] == '1':
        return '0-small'
    else:
        raise Exception('Unknown MaximumSize Exception with size: ' + s)

def str_to_minimum_size(s):
    if(pd.isna(s)):
        return 'no-data'
    
    if s[3] == '1':
        return '0-small'
    elif s[2] == '1':
        return '1-large'
    elif s[1] == '1':
        return '2-very large'
    elif s[0] == '1':
        return '3-historic'
    else:
        raise Exception('Unknown MinimumSize Exception with size: ' + s)

    
def row_to_avy_problem(row):
    num_problems = row['problems']
    if pd.isna(num_problems):
        return
    
    
    for i in range(0, int(num_problems)):
        try:
            problem_type = num_to_problem(row['problem_' + str(i)])
       
            #print('processing problem type: ' + problem_type)
            row[problem_type + '_Likelihood'] = num_to_likelihood(row['likelihood_' + str(i)])
            row[problem_type + '_MaximumSize'] = str_to_maximum_size(row['size_' + str(i)])
            row[problem_type + '_MinimumSize'] = str_to_minimum_size(row['size_' + str(i)])

            if pd.isna(row['aspect_elev_' + str(i)]):
                row[problem_type + '_OctagonBelowTreelineNorth'] = 'no-data'
                row[problem_type + '_OctagonNearTreelineNorth'] = 'no-data'
                row[problem_type + '_OctagonAboveTreelineNorth'] = 'no-data'
                row[problem_type + '_OctagonBelowTreelineNorthEast'] = 'no-data'
                row[problem_type + '_OctagonNearTreelineNorthEast'] = 'no-data'
                row[problem_type + '_OctagonAboveTreelineNorthEast'] = 'no-data'
                row[problem_type + '_OctagonBelowTreelineEast'] = 'no-data'
                row[problem_type + '_OctagonNearTreelineEast'] = 'no-data'
                row[problem_type + '_OctagonAboveTreelineEast'] = 'no-data'
                row[problem_type + '_OctagonBelowTreelineSouthEast'] = 'no-data'
                row[problem_type + '_OctagonNearTreelineSouthEast'] = 'no-data'
                row[problem_type + '_OctagonAboveTreelineSouthEast'] = 'no-data'
                row[problem_type + '_OctagonBelowTreelineSouth'] = 'no-data'
                row[problem_type + '_OctagonNearTreelineSouth'] = 'no-data'
                row[problem_type + '_OctagonAboveTreelineSouth'] = 'no-data'
                row[problem_type + '_OctagonBelowTreelineSouthWest'] = 'no-data'
                row[problem_type + '_OctagonNearTreelineSouthWest'] = 'no-data'
                row[problem_type + '_OctagonAboveTreelineSouthWest'] = 'no-data'
                row[problem_type + '_OctagonBelowTreelineWest'] = 'no-data'
                row[problem_type + '_OctagonNearTreelineWest'] = 'no-data'
                row[problem_type + '_OctagonAboveTreelineWest'] = 'no-data'
                row[problem_type + '_OctagonBelowTreelineNorthWest'] = 'no-data'
                row[problem_type + '_OctagonNearTreelineNorthWest'] = 'no-data'
                row[problem_type + '_OctagonAboveTreelineNorthWest'] = 'no-data'
            else:  
                row[problem_type + '_OctagonBelowTreelineNorth'] = row['aspect_elev_' + str(i)][0]
                row[problem_type + '_OctagonNearTreelineNorth'] = row['aspect_elev_' + str(i)][1]
                row[problem_type + '_OctagonAboveTreelineNorth'] = row['aspect_elev_' + str(i)][2]
                row[problem_type + '_OctagonBelowTreelineNorthEast'] = row['aspect_elev_' + str(i)][3]
                row[problem_type + '_OctagonNearTreelineNorthEast'] = row['aspect_elev_' + str(i)][4]
                row[problem_type + '_OctagonAboveTreelineNorthEast'] = row['aspect_elev_' + str(i)][5]
                row[problem_type + '_OctagonBelowTreelineEast'] = row['aspect_elev_' + str(i)][6]
                row[problem_type + '_OctagonNearTreelineEast'] = row['aspect_elev_' + str(i)][7]
                row[problem_type + '_OctagonAboveTreelineEast'] = row['aspect_elev_' + str(i)][8]
                row[problem_type + '_OctagonBelowTreelineSouthEast'] = row['aspect_elev_' + str(i)][9]
                row[problem_type + '_OctagonNearTreelineSouthEast'] = row['aspect_elev_' + str(i)][10]
                row[problem_type + '_OctagonAboveTreelineSouthEast'] = row['aspect_elev_' + str(i)][11]
                row[problem_type + '_OctagonBelowTreelineSouth'] = row['aspect_elev_' + str(i)][12]
                row[problem_type + '_OctagonNearTreelineSouth'] = row['aspect_elev_' + str(i)][13]
                row[problem_type + '_OctagonAboveTreelineSouth'] = row['aspect_elev_' + str(i)][14]
                row[problem_type + '_OctagonBelowTreelineSouthWest'] = row['aspect_elev_' + str(i)][15]
                row[problem_type + '_OctagonNearTreelineSouthWest'] = row['aspect_elev_' + str(i)][16]
                row[problem_type + '_OctagonAboveTreelineSouthWest'] = row['aspect_elev_' + str(i)][17]
                row[problem_type + '_OctagonBelowTreelineWest'] = row['aspect_elev_' + str(i)][18]
                row[problem_type + '_OctagonNearTreelineWest'] = row['aspect_elev_' + str(i)][19]
                row[problem_type + '_OctagonAboveTreelineWest'] = row['aspect_elev_' + str(i)][20]
                row[problem_type + '_OctagonBelowTreelineNorthWest'] = row['aspect_elev_' + str(i)][21]
                row[problem_type + '_OctagonNearTreelineNorthWest'] = row['aspect_elev_' + str(i)][22]
                row[problem_type + '_OctagonAboveTreelineNorthWest'] = row['aspect_elev_' + str(i)][23]
        except:
            print("format exception on bc_avo_fx_id: " + str(row['bc_avo_fx_id']) + ' skipping')
    return row
        
        
    
        

In [79]:
tmp = df.apply(row_to_avy_problem, axis=1)

format exception on bc_avo_fx_id: 284 skipping
format exception on bc_avo_fx_id: 305 skipping
format exception on bc_avo_fx_id: 319 skipping
format exception on bc_avo_fx_id: 325 skipping
format exception on bc_avo_fx_id: 326 skipping
format exception on bc_avo_fx_id: 329 skipping
format exception on bc_avo_fx_id: 338 skipping
format exception on bc_avo_fx_id: 342 skipping
format exception on bc_avo_fx_id: 495 skipping
format exception on bc_avo_fx_id: 508 skipping
format exception on bc_avo_fx_id: 513 skipping
format exception on bc_avo_fx_id: 693 skipping
format exception on bc_avo_fx_id: 693 skipping
format exception on bc_avo_fx_id: 699 skipping
format exception on bc_avo_fx_id: 714 skipping
format exception on bc_avo_fx_id: 714 skipping
format exception on bc_avo_fx_id: 721 skipping
format exception on bc_avo_fx_id: 722 skipping
format exception on bc_avo_fx_id: 725 skipping
format exception on bc_avo_fx_id: 889 skipping
format exception on bc_avo_fx_id: 1072 skipping
format excep

In [80]:
#tmp.to_csv("tmp.csv")

In [81]:
tmp.rename(columns={'name': 'Region', 'date_issued': 'PublishedDateTime', 'summary': 'Day1DetailedForecast'}, inplace=True)



In [82]:
tmp['ForecastUrl'] = ''
tmp['BottomLineSummary'] = ''
tmp['Day1Warning'] = ''
tmp['Day1WarningEnd'] = ''
tmp['Day1WarningText'] = ''
tmp['Day2DetailedForecast'] = ''
tmp['Day2Warning'] = ''
tmp['Day2WarningEnd'] = ''
tmp['Day2WarningText'] = ''
tmp['SpecialStatement'] = ''
tmp['UnifiedRegion'] = tmp['Region']

In [83]:

tmp['Day1Date'] = tmp['PublishedDateTime'].dt.strftime('%Y%m%d')

In [15]:
#tmp.to_csv("tmp.csv")

In [84]:
#filter to the latest forecast for the date (to remove)
#currently just taking the last forecast for the day but this could be improved to differentiate between updated and new forecasts
#also end of season forecast will be overwritten by placeholder forecast
tmp['maxdate'] = tmp.groupby(['Region', 'Day1Date'])['PublishedDateTime'].transform('max')

In [85]:
tmp['PublishedDateTime'] = pd.to_datetime(tmp['PublishedDateTime'])
tmp['maxdate'] = pd.to_datetime(tmp['maxdate'])


In [86]:
tmp = tmp[tmp['PublishedDateTime'] == tmp['maxdate']]

In [87]:
tmp['Day1Date'].head()

0    20131223
1    20131223
2    20131223
3    20131223
4    20131223
Name: Day1Date, dtype: object

In [88]:
#dump the critical points
tmp[['Region', 'lat', 'lon']].groupby(['Region', 'lat', 'lon']).max()

,,
Region,lat,lon
Aspen,39.29493,-106.92443
Front Range,40.14328,-105.84075
Grand Mesa,39.07342,-107.97518
Gunnison,38.83853,-106.57729
Northern San Juan,37.97170,-107.29020
Sangre de Cristo,37.73237,-105.33148
Sawatch Range,38.91241,-106.35864
Southern San Juan,37.53723,-106.80449
Steamboat & Flat Tops,40.45217,-106.66901


In [89]:
#final column list
cols = [
    'Region',
    'UnifiedRegion',
    'PublishedDateTime',
    'Day1Date',
    'SpecialStatement',
    'BottomLineSummary',
    'ForecastUrl',
    'Day1DangerAboveTreeline',
    'Day1DangerNearTreeline',
    'Day1DangerBelowTreeline',
    'Day1DetailedForecast',
    'Day1Warning',
    'Day1WarningEnd',
    'Day1WarningText',
    'Day2DangerAboveTreeline',
    'Day2DangerNearTreeline',
    'Day2DangerBelowTreeline',
    'Day2DetailedForecast',
    'Day2Warning',
    'Day2WarningEnd',
    'Day2WarningText',
    'Cornices_Likelihood',
    'Cornices_MaximumSize',
    'Cornices_MinimumSize',
    'Cornices_OctagonAboveTreelineEast',
    'Cornices_OctagonAboveTreelineNorth',
    'Cornices_OctagonAboveTreelineNorthEast',
    'Cornices_OctagonAboveTreelineNorthWest',
    'Cornices_OctagonAboveTreelineSouth',
    'Cornices_OctagonAboveTreelineSouthEast',
    'Cornices_OctagonAboveTreelineSouthWest',
    'Cornices_OctagonAboveTreelineWest',
    'Cornices_OctagonNearTreelineEast',
    'Cornices_OctagonNearTreelineNorth',
    'Cornices_OctagonNearTreelineNorthEast',
    'Cornices_OctagonNearTreelineNorthWest',
    'Cornices_OctagonNearTreelineSouth',
    'Cornices_OctagonNearTreelineSouthEast',
    'Cornices_OctagonNearTreelineSouthWest',
    'Cornices_OctagonNearTreelineWest',
    'Cornices_OctagonBelowTreelineEast',
    'Cornices_OctagonBelowTreelineNorth',
    'Cornices_OctagonBelowTreelineNorthEast',
    'Cornices_OctagonBelowTreelineNorthWest',
    'Cornices_OctagonBelowTreelineSouth',
    'Cornices_OctagonBelowTreelineSouthEast',
    'Cornices_OctagonBelowTreelineSouthWest',
    'Cornices_OctagonBelowTreelineWest',
    'Glide_Likelihood',
    'Glide_MaximumSize',
    'Glide_MinimumSize',
    'Glide_OctagonAboveTreelineEast',
    'Glide_OctagonAboveTreelineNorth',
    'Glide_OctagonAboveTreelineNorthEast',
    'Glide_OctagonAboveTreelineNorthWest',
    'Glide_OctagonAboveTreelineSouth',
    'Glide_OctagonAboveTreelineSouthEast',
    'Glide_OctagonAboveTreelineSouthWest',
    'Glide_OctagonAboveTreelineWest',
    'Glide_OctagonNearTreelineEast',
    'Glide_OctagonNearTreelineNorth',
    'Glide_OctagonNearTreelineNorthEast',
    'Glide_OctagonNearTreelineNorthWest',
    'Glide_OctagonNearTreelineSouth',
    'Glide_OctagonNearTreelineSouthEast',
    'Glide_OctagonNearTreelineSouthWest',
    'Glide_OctagonNearTreelineWest',
    'Glide_OctagonBelowTreelineEast',
    'Glide_OctagonBelowTreelineNorth',
    'Glide_OctagonBelowTreelineNorthEast',
    'Glide_OctagonBelowTreelineNorthWest',
    'Glide_OctagonBelowTreelineSouth',
    'Glide_OctagonBelowTreelineSouthEast',
    'Glide_OctagonBelowTreelineSouthWest',
    'Glide_OctagonBelowTreelineWest',
    'LooseDry_Likelihood',
    'LooseDry_MaximumSize',
    'LooseDry_MinimumSize',
    'LooseDry_OctagonAboveTreelineEast',
    'LooseDry_OctagonAboveTreelineNorth',
    'LooseDry_OctagonAboveTreelineNorthEast',
    'LooseDry_OctagonAboveTreelineNorthWest',
    'LooseDry_OctagonAboveTreelineSouth',
    'LooseDry_OctagonAboveTreelineSouthEast',
    'LooseDry_OctagonAboveTreelineSouthWest',
    'LooseDry_OctagonAboveTreelineWest',
    'LooseDry_OctagonNearTreelineEast',
    'LooseDry_OctagonNearTreelineNorth',
    'LooseDry_OctagonNearTreelineNorthEast',
    'LooseDry_OctagonNearTreelineNorthWest',
    'LooseDry_OctagonNearTreelineSouth',
    'LooseDry_OctagonNearTreelineSouthEast',
    'LooseDry_OctagonNearTreelineSouthWest',
    'LooseDry_OctagonNearTreelineWest',
    'LooseDry_OctagonBelowTreelineEast',
    'LooseDry_OctagonBelowTreelineNorth',
    'LooseDry_OctagonBelowTreelineNorthEast',
    'LooseDry_OctagonBelowTreelineNorthWest',
    'LooseDry_OctagonBelowTreelineSouth',
    'LooseDry_OctagonBelowTreelineSouthEast',
    'LooseDry_OctagonBelowTreelineSouthWest',
    'LooseDry_OctagonBelowTreelineWest',
    'LooseWet_Likelihood',
    'LooseWet_MaximumSize',
    'LooseWet_MinimumSize',
    'LooseWet_OctagonAboveTreelineEast',
    'LooseWet_OctagonAboveTreelineNorth',
    'LooseWet_OctagonAboveTreelineNorthEast',
    'LooseWet_OctagonAboveTreelineNorthWest',
    'LooseWet_OctagonAboveTreelineSouth',
    'LooseWet_OctagonAboveTreelineSouthEast',
    'LooseWet_OctagonAboveTreelineSouthWest',
    'LooseWet_OctagonAboveTreelineWest',
    'LooseWet_OctagonNearTreelineEast',
    'LooseWet_OctagonNearTreelineNorth',
    'LooseWet_OctagonNearTreelineNorthEast',
    'LooseWet_OctagonNearTreelineNorthWest',
    'LooseWet_OctagonNearTreelineSouth',
    'LooseWet_OctagonNearTreelineSouthEast',
    'LooseWet_OctagonNearTreelineSouthWest',
    'LooseWet_OctagonNearTreelineWest',
    'LooseWet_OctagonBelowTreelineEast',
    'LooseWet_OctagonBelowTreelineNorth',
    'LooseWet_OctagonBelowTreelineNorthEast',
    'LooseWet_OctagonBelowTreelineNorthWest',
    'LooseWet_OctagonBelowTreelineSouth',
    'LooseWet_OctagonBelowTreelineSouthEast',
    'LooseWet_OctagonBelowTreelineSouthWest',
    'LooseWet_OctagonBelowTreelineWest',
    'PersistentSlab_Likelihood',
    'PersistentSlab_MaximumSize',
    'PersistentSlab_MinimumSize',
    'PersistentSlab_OctagonAboveTreelineEast',
    'PersistentSlab_OctagonAboveTreelineNorth',
    'PersistentSlab_OctagonAboveTreelineNorthEast',
    'PersistentSlab_OctagonAboveTreelineNorthWest',
    'PersistentSlab_OctagonAboveTreelineSouth',
    'PersistentSlab_OctagonAboveTreelineSouthEast',
    'PersistentSlab_OctagonAboveTreelineSouthWest',
    'PersistentSlab_OctagonAboveTreelineWest',
    'PersistentSlab_OctagonNearTreelineEast',
    'PersistentSlab_OctagonNearTreelineNorth',
    'PersistentSlab_OctagonNearTreelineNorthEast',
    'PersistentSlab_OctagonNearTreelineNorthWest',
    'PersistentSlab_OctagonNearTreelineSouth',
    'PersistentSlab_OctagonNearTreelineSouthEast',
    'PersistentSlab_OctagonNearTreelineSouthWest',
    'PersistentSlab_OctagonNearTreelineWest',
    'PersistentSlab_OctagonBelowTreelineEast',
    'PersistentSlab_OctagonBelowTreelineNorth',
    'PersistentSlab_OctagonBelowTreelineNorthEast',
    'PersistentSlab_OctagonBelowTreelineNorthWest',
    'PersistentSlab_OctagonBelowTreelineSouth',
    'PersistentSlab_OctagonBelowTreelineSouthEast',
    'PersistentSlab_OctagonBelowTreelineSouthWest',
    'PersistentSlab_OctagonBelowTreelineWest',
    'DeepPersistentSlab_Likelihood',
    'DeepPersistentSlab_MaximumSize',
    'DeepPersistentSlab_MinimumSize',
    'DeepPersistentSlab_OctagonAboveTreelineEast',
    'DeepPersistentSlab_OctagonAboveTreelineNorth',
    'DeepPersistentSlab_OctagonAboveTreelineNorthEast',
    'DeepPersistentSlab_OctagonAboveTreelineNorthWest',
    'DeepPersistentSlab_OctagonAboveTreelineSouth',
    'DeepPersistentSlab_OctagonAboveTreelineSouthEast',
    'DeepPersistentSlab_OctagonAboveTreelineSouthWest',
    'DeepPersistentSlab_OctagonAboveTreelineWest',
    'DeepPersistentSlab_OctagonNearTreelineEast',
    'DeepPersistentSlab_OctagonNearTreelineNorth',
    'DeepPersistentSlab_OctagonNearTreelineNorthEast',
    'DeepPersistentSlab_OctagonNearTreelineNorthWest',
    'DeepPersistentSlab_OctagonNearTreelineSouth',
    'DeepPersistentSlab_OctagonNearTreelineSouthEast',
    'DeepPersistentSlab_OctagonNearTreelineSouthWest',
    'DeepPersistentSlab_OctagonNearTreelineWest',
    'DeepPersistentSlab_OctagonBelowTreelineEast',
    'DeepPersistentSlab_OctagonBelowTreelineNorth',
    'DeepPersistentSlab_OctagonBelowTreelineNorthEast',
    'DeepPersistentSlab_OctagonBelowTreelineNorthWest',
    'DeepPersistentSlab_OctagonBelowTreelineSouth',
    'DeepPersistentSlab_OctagonBelowTreelineSouthEast',
    'DeepPersistentSlab_OctagonBelowTreelineSouthWest',
    'DeepPersistentSlab_OctagonBelowTreelineWest',
    'StormSlabs_Likelihood',
    'StormSlabs_MaximumSize',
    'StormSlabs_MinimumSize',
    'StormSlabs_OctagonAboveTreelineEast',
    'StormSlabs_OctagonAboveTreelineNorth',
    'StormSlabs_OctagonAboveTreelineNorthEast',
    'StormSlabs_OctagonAboveTreelineNorthWest',
    'StormSlabs_OctagonAboveTreelineSouth',
    'StormSlabs_OctagonAboveTreelineSouthEast',
    'StormSlabs_OctagonAboveTreelineSouthWest',
    'StormSlabs_OctagonAboveTreelineWest',
    'StormSlabs_OctagonNearTreelineEast',
    'StormSlabs_OctagonNearTreelineNorth',
    'StormSlabs_OctagonNearTreelineNorthEast',
    'StormSlabs_OctagonNearTreelineNorthWest',
    'StormSlabs_OctagonNearTreelineSouth',
    'StormSlabs_OctagonNearTreelineSouthEast',
    'StormSlabs_OctagonNearTreelineSouthWest',
    'StormSlabs_OctagonNearTreelineWest',
    'StormSlabs_OctagonBelowTreelineEast',
    'StormSlabs_OctagonBelowTreelineNorth',
    'StormSlabs_OctagonBelowTreelineNorthEast',
    'StormSlabs_OctagonBelowTreelineNorthWest',
    'StormSlabs_OctagonBelowTreelineSouth',
    'StormSlabs_OctagonBelowTreelineSouthEast',
    'StormSlabs_OctagonBelowTreelineSouthWest',
    'StormSlabs_OctagonBelowTreelineWest',
    'WetSlabs_Likelihood',
    'WetSlabs_MaximumSize',
    'WetSlabs_MinimumSize',
    'WetSlabs_OctagonAboveTreelineEast',
    'WetSlabs_OctagonAboveTreelineNorth',
    'WetSlabs_OctagonAboveTreelineNorthEast',
    'WetSlabs_OctagonAboveTreelineNorthWest',
    'WetSlabs_OctagonAboveTreelineSouth',
    'WetSlabs_OctagonAboveTreelineSouthEast',
    'WetSlabs_OctagonAboveTreelineSouthWest',
    'WetSlabs_OctagonAboveTreelineWest',
    'WetSlabs_OctagonNearTreelineEast',
    'WetSlabs_OctagonNearTreelineNorth',
    'WetSlabs_OctagonNearTreelineNorthEast',
    'WetSlabs_OctagonNearTreelineNorthWest',
    'WetSlabs_OctagonNearTreelineSouth',
    'WetSlabs_OctagonNearTreelineSouthEast',
    'WetSlabs_OctagonNearTreelineSouthWest',
    'WetSlabs_OctagonNearTreelineWest',
    'WetSlabs_OctagonBelowTreelineEast',
    'WetSlabs_OctagonBelowTreelineNorth',
    'WetSlabs_OctagonBelowTreelineNorthEast',
    'WetSlabs_OctagonBelowTreelineNorthWest',
    'WetSlabs_OctagonBelowTreelineSouth',
    'WetSlabs_OctagonBelowTreelineSouthEast',
    'WetSlabs_OctagonBelowTreelineSouthWest',
    'WetSlabs_OctagonBelowTreelineWest',
    'WindSlab_Likelihood',
    'WindSlab_MaximumSize',
    'WindSlab_MinimumSize',
    'WindSlab_OctagonAboveTreelineEast',
    'WindSlab_OctagonAboveTreelineNorth',
    'WindSlab_OctagonAboveTreelineNorthEast',
    'WindSlab_OctagonAboveTreelineNorthWest',
    'WindSlab_OctagonAboveTreelineSouth',
    'WindSlab_OctagonAboveTreelineSouthEast',
    'WindSlab_OctagonAboveTreelineSouthWest',
    'WindSlab_OctagonAboveTreelineWest',
    'WindSlab_OctagonNearTreelineEast',
    'WindSlab_OctagonNearTreelineNorth',
    'WindSlab_OctagonNearTreelineNorthEast',
    'WindSlab_OctagonNearTreelineNorthWest',
    'WindSlab_OctagonNearTreelineSouth',
    'WindSlab_OctagonNearTreelineSouthEast',
    'WindSlab_OctagonNearTreelineSouthWest',
    'WindSlab_OctagonNearTreelineWest',
    'WindSlab_OctagonBelowTreelineEast',
    'WindSlab_OctagonBelowTreelineNorth',
    'WindSlab_OctagonBelowTreelineNorthEast',
    'WindSlab_OctagonBelowTreelineNorthWest',
    'WindSlab_OctagonBelowTreelineSouth',
    'WindSlab_OctagonBelowTreelineSouthEast',
    'WindSlab_OctagonBelowTreelineSouthWest',
    'WindSlab_OctagonBelowTreelineWest'
]


In [90]:
len(tmp)

7306

In [91]:
finalDf = tmp[cols]

In [92]:
finalDf.replace(np.nan, 'no-data', inplace=True)

C:\Users\scott\Anaconda3\envs\cntkpython36\lib\site-packages\pandas\core\frame.py:3798: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [93]:
finalDf[:20]

,Region,UnifiedRegion,PublishedDateTime,Day1Date,SpecialStatement,BottomLineSummary,ForecastUrl,Day1DangerAboveTreeline,Day1DangerNearTreeline,Day1DangerBelowTreeline,...,WindSlab_OctagonNearTreelineSouthWest,WindSlab_OctagonNearTreelineWest,WindSlab_OctagonBelowTreelineEast,WindSlab_OctagonBelowTreelineNorth,WindSlab_OctagonBelowTreelineNorthEast,WindSlab_OctagonBelowTreelineNorthWest,WindSlab_OctagonBelowTreelineSouth,WindSlab_OctagonBelowTreelineSouthEast,WindSlab_OctagonBelowTreelineSouthWest,WindSlab_OctagonBelowTreelineWest
0,Grand Mesa,Grand Mesa,2013-12-23 16:16:55,20131223,,,,no-data,Moderate,Low,...,0,0,0,0,0,0,0,0,0,0
1,Sangre de Cristo,Sangre de Cristo,2013-12-23 16:19:20,20131223,,,,no-data,no-data,no-data,...,no-data,no-data,no-data,no-data,no-data,no-data,no-data,no-data,no-data,no-data
2,Steamboat & Flat Tops,Steamboat & Flat Tops,2013-12-23 16:23:30,20131223,,,,Considerable,Considerable,Moderate,...,0,0,0,0,0,0,0,0,0,0
3,Front Range,Front Range,2013-12-23 16:26:13,20131223,,,,Considerable,Considerable,Low,...,0,0,0,0,0,0,0,0,0,0
4,Vail & Summit County,Vail & Summit County,2013-12-23 16:29:09,20131223,,,,Considerable,Considerable,Low,...,0,0,0,0,0,0,0,0,0,0
5,Sawatch Range,Sawatch Range,2013-12-23 16:33:09,20131223,,,,Moderate,Moderate,Low,...,0,0,0,0,0,0,0,0,0,0
6,Aspen,Aspen,2013-12-23 16:37:11,20131223,,,,Moderate,Moderate,Low,...,0,0,0,0,0,0,0,0,0,0
7,Northern San Juan,Northern San Juan,2013-12-23 16:40:41,20131223,,,,Moderate,Moderate,Low,...,1,0,0,0,0,0,0,0,0,0
8,Southern San Juan,Southern San Juan,2013-12-23 16:43:40,20131223,,,,Moderate,Moderate,Low,...,no-data,no-data,no-data,no-data,no-data,no-data,no-data,no-data,no-data,no-data
9,Gunnison,Gunnison,2013-12-23 16:56:42,20131223,,,,Moderate,Moderate,Low,...,1,0,0,0,0,0,0,0,0,0


In [95]:
finalDf.to_csv('CleanedForecastsCAIC.V1.csv', index=False, date_format='%Y%m%d %H:00')